<a href="https://colab.research.google.com/github/opensim-org/opensim-models/blob/master/notebooks/tutorials/Tutorial_3_Load_Save_Models_and_Accessing_Modifying_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Load/Save Models and Accessing/Modifying Data.ipynb

## 3.1. Objectives

**Purpose**

The purpose of this tutorial is to learn to load and save models in [OpenSim](https://opensim.stanford.edu/) [[1]](https://doi.org/10.1109/TBME.2007.901024), as well as to learn to access and modify the data in those models. In this tutorial you will:

*   Learn to load an existing model.
*   Access and modify the information inside of a model.
*   Save your models.

**Format**

In this tutorial, you will download a model from internet and load it into OpenSim. Then, you will access and print information from the model. Finally, you will modify the model and save and download it to your computer. 

## 3.2. Setup Conda and OpenSim

First, set up the environment by executing the following cell (See ***Tutorial 1: Introduction to OpenSim in Colab***).

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:35
🔁 Restarting kernel...


Now, let's install the OpenSim conda package.

In [ ]:
!conda install -c opensim-org opensim

/bin/bash: conda: command not found


## 3.3. Load an existing model and explore the data

In this section you will download a model from the internet, load it into OpenSim, and explore some of its properties. The model consists on a double pendulum, and it is hosted in a GitHub repository.

First, let's download the model from the GitHub repository.

In [ ]:
import requests

# TODO Host tutorial models elsewhere?
# Download the model from the GitHub repository.
url = 'https://raw.githubusercontent.com/opensim-org/opensim-core/a01cf545485e8bc1f6e50b29ddeab229677de2c5/OpenSim/Simulation/Test/double_pendulum.osim'
req = requests.get(url, allow_redirects=True)
open('double_pendulum.osim', 'wb').write(req.content)

14435

Now, let's import opensim, load the model, and print some metadata.

In [ ]:
import opensim as osim

# Load the model.
double_pendulum = osim.Model('double_pendulum.osim')

# Print metadata.
print("Name of the model:", double_pendulum.getName())
print("Author:", double_pendulum.get_credits())
print("Publications:", double_pendulum.get_publications())
print("Length Unit:", double_pendulum.get_length_units())
print("Force Unit:", double_pendulum.get_force_units())
print("Gravity:", double_pendulum.get_gravity())

Name of the model: double_pendulum
Author: Ajay Seth
Publications: For testing
Length Unit: meters
Force Unit: N
Gravity: ~[0,-9.80665,0]


Let's explore some of them model's body properties. Since it is a double pendulum, we expect the model to have two bodies.

In [ ]:
# Print number of bodies.
print("Num Bodies:", double_pendulum.getNumBodies())
print()

# For each body, print some information like its name or mass.
for body in double_pendulum.getBodySet():
  print("Body Name:", body.getName())
  print("Body Absolute Path:", body.getAbsolutePathString())
  print("Body Mass:", body.getMass())
  print("Body Mass Center:", body.getMassCenter())
  print("Body Input Names:", body.getInputNames())
  print()

Num Bodies: 2

Body Name: rod1
Body Absolute Path: /bodyset/rod1
Body Mass: 1.0
Body Mass Center: ~[0,0.2,0]
Body Input Names: ()

Body Name: rod2
Body Absolute Path: /bodyset/rod2
Body Mass: 1.0
Body Mass Center: ~[0,0,0]
Body Input Names: ()



Let's explore some of the model's joint properties. Again, we expect to have two joints. The first one from the ground to rod1, so it is fixed and doesn't fall when you simulate it; and the second one from rod1 to rod2, linking the two bodies of the double pendulum.

In [ ]:
# Print number of joints.
print("Num Joints:", double_pendulum.getNumJoints())
print()

# For each joint, print some information like its name or components.
for joint in double_pendulum.getJointSet():
  print("Joint Name:", joint.getName())
  print("Joint Absolute Path:", joint.getAbsolutePathString())
  print("Components:")
  for component in joint.getComponentsList():
    print("  Component Name:", component.getName())
    print("  Component Absolute Path:", component.getAbsolutePathString())
  print()

Num Joints: 2

Joint Name: pin1
Joint Absolute Path: /jointset/pin1
Components:
  Component Name: q1
  Component Absolute Path: /jointset/pin1/q1
  Component Name: ground_offset
  Component Absolute Path: /jointset/pin1/ground_offset
  Component Name: frame_geometry
  Component Absolute Path: /jointset/pin1/ground_offset/frame_geometry
  Component Name: wrapobjectset
  Component Absolute Path: /jointset/pin1/ground_offset/wrapobjectset
  Component Name: rod1_offset
  Component Absolute Path: /jointset/pin1/rod1_offset
  Component Name: frame_geometry
  Component Absolute Path: /jointset/pin1/rod1_offset/frame_geometry
  Component Name: wrapobjectset
  Component Absolute Path: /jointset/pin1/rod1_offset/wrapobjectset

Joint Name: pin2
Joint Absolute Path: /jointset/pin2
Components:
  Component Name: q2
  Component Absolute Path: /jointset/pin2/q2
  Component Name: rod1_offset
  Component Absolute Path: /jointset/pin2/rod1_offset
  Component Name: frame_geometry
  Component Absolute Path

Finally, let's explore the coordinates of the model.

In [ ]:
# Print number of coordinates.
print("Num Coordinates:", double_pendulum.getNumCoordinates())
print()

# For each coordinate, print some information like its name or motion type.
for coordinate in double_pendulum.getCoordinateSet():
  print("  Coordinate Name:", coordinate.getName())
  print("  Coordinate Absolute Path:", coordinate.getAbsolutePathString())
  print("  Coordinate Motion Type:", coordinate.getMotionType())
  
  print()

Num Coordinates: 2

  Coordinate Name: q1
  Coordinate Absolute Path: /jointset/pin1/q1
  Coordinate Motion Type: 1

  Coordinate Name: q2
  Coordinate Absolute Path: /jointset/pin2/q2
  Coordinate Motion Type: 1



## 3.4. Modify the model and save it

Let's modify some of the model's properties. For example, let's set the gravity of mars to the model, and double the mass of the bodies. After that, we want to save the changes, so we will save the model into a new file.

First, let's modify the gravity of the model to mars' gravity (-3.77), and let's double the mass of the bodies.

In [ ]:
import numpy as np

# Set gravity of mars and print it.
double_pendulum.set_gravity(osim.Vec3(0, -3.77, 0))
print("New Gravity:", double_pendulum.get_gravity())
print()

# For each body, print its name, set mass as 2Kg, and print new mass.
for body in double_pendulum.getBodyList():
  print("Body Name:", body.getName())
  body.setMass(2.0)
  print("Body New Mass:", body.getMass())
  print()

New Gravity: ~[0,-3.77,0]

Body Name: rod1
Body New Mass: 2.0

Body Name: rod2
Body New Mass: 2.0



Now, let's save the model into an `osim` file.

In [ ]:
double_pendulum.printToXML('heavy-double-pendulum-mars.osim')

True

The file can be downloaded either by `right-clicking` on it at the left and clicking `download` (if the file is not in the tree at the left, you may need to `right-click` and `refresh`).

Alternativelly, you can download the file programatically as shown in the following cell.

In [ ]:
from google.colab import files

files.download('heavy-double-pendulum-mars.osim') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3.5. Conclussion

In this tutorial you have downloaded a double pendulum model and loaded it into OpenSim. Then you have explored some of its properties and components, and finally you have modified it and saved it into an osim file, that you can download to your computer.

## 3.6. Useful Links





> **OpenSim Website:** https://opensim.stanford.edu/
>
> **OpenSim API Documentation:** https://simtk.org/api_docs/opensim/api_docs/
> 
> **OpenSim Creator Website:** https://opensimcreator.com/
> 
> **SimTK Website:** https://simtk.org/projects/opensim
> 
> **Biomecanics of Movement's Course:** https://www.youtube.com/channel/UCDNGy0KKNLQ-ztcL5h2Z6zA

## 3.7. References




> [1].   Delp, S. L., Anderson, F. C., Arnold, A. S., Loan, P., Habib, A., John, C. T., Guendelman, E., & Thelen, D. G. (2007). **OpenSim: open-source software to create and analyze dynamic simulations of movement.** *IEEE Transactions on Bio-Medical Engineering*, 54(11), 1940–1950. https://doi.org/10.1109/TBME.2007.901024